In [51]:
from datetime import datetime
import sys
import time
import sqlite3
import pandas as pd
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 
import gensim
import re
from gensim.summarization.mz_entropy import mz_keywords
import time
import spacy

In [52]:
#load
data = pd.read_parquet("data.parquet", engine="fastparquet")
original = data.copy(deep=True)

In [53]:
def get_cat(row):
    if ("ENG - Software Engineering" in row["targeted_disciplines"]) and ("MATH - Computer Science" in row["targeted_disciplines"]):
        return True
    else:
        return False
    
def merge_duplicates(df):
    df = df.drop_duplicates(subset=['organization', 'job_title'], keep="first") 
    df = df.drop_duplicates(subset=['job_responsibilities', 'job_summary', 'required_skills'], keep="first")
    return df

def clean(x):
    newstr = strip_multiple_whitespaces(x)
    print(newstr)
    return newstr

def preprocess(x):
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords, strip_multiple_whitespaces]
    x = re.sub(r"http\S+", "", x)
    x = preprocess_string(x, CUSTOM_FILTERS)
    x = " ".join(x)
    x = re.sub(r'\d+', '', x)
    return x

In [54]:
#Clean data
data["job_responsibilities"] = data["job_responsibilities"].map(lambda x: preprocess(x))
data["job_summary"] = data["job_summary"].map(lambda x: preprocess(x))
data["required_skills"] = data["required_skills"].map(lambda x: preprocess(x))
data = merge_duplicates(data)
m = data.apply(get_cat, axis=1)
data = data[m]

In [55]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        keywords = []
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            keywords.append([key, value])
            if i > number:
                break
        return keywords
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
#Entire resume comparison
resume = "Worked on proof-of-concept projects to demonstrate ML feasibility in key supply chain areas. Pre-processed time series data, trained forecasting models, and evaluated predictions. Mentored other interns on data science libraries, Jupyter, natural language processing, and ML best practices. Secured over $15k+ worth of sponsorship for over 6 events (1200+ attendees combined) since inception. \
Created fully functional platform to collect hackathon data for analytics. Hosted one of Canada’s first game development hackathons with Red Bull Canada: redbull.com/adrenalan. Experimented with state of the art NLP techniques to detect type of toxicity in online comments. \
Experimented with LSTMs, word embeddings (Glove, FastText), and techniques such as sequence bucketing. \
Fine-tuned and ensembled Bidirectional Encoder Representations from Transformers (BERT). \
Integrated an “auto-swipe” feature into Tinder based on real-time brain EEG data collected by the Muse. \
Tested on participants at Hack The 6ix 2017 and achieved a 75% accuracy. \
Knowledgeable in: Git, Docker, Python (Pandas, Numpy, Scikit-Learn), Keras, Pytorch, Java, Javascript, VueJS, Firebase \
"

In [ ]:
tr4w = TextRank4Keyword()
tr4w.analyze(resume, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
keywords = tr4w.get_keywords(50)

In [ ]:
def score(keywords, job):
    tr4w.analyze(job, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
    job_keywords = tr4w.get_keywords(50)
    score = 0
    for w1 in keywords:
        for w2 in job_keywords:
            if w1[0] == w2[0]:
                score += w1[1]
    return score

In [ ]:

start = time.time()
scores = []
for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
    doc = doc1 + " " + doc2 + " " + doc3
    scores.append((i, score(keywords, doc)))
print(time.time() - start)

In [ ]:
scores = sorted(scores,key=lambda x: x[1], reverse=True)

In [ ]:
for job in scores[:50]:
    X = job[0]
    print("ID: ", X)
    print(original["job_title"].values[X], ", ", original["organization"].values[X])
    print(original["job_responsibilities"].values[X])
    print(original["job_summary"].values[X])
    print(original["required_skills"].values[X])
    print("================================")
    print()

In [56]:
#Section comparison
resume1 = "Worked on proof-of-concept projects to demonstrate ML feasibility in key supply chain areas. Pre-processed time series data, trained forecasting models, and evaluated predictions. Mentored other interns on data science libraries, Jupyter, natural language processing, and ML best practices."
resume2 = "Secured over $15k+ worth of sponsorship for over 6 events (1200+ attendees combined) since inception. Created fully functional platform to collect hackathon data for analytics. Hosted one of Canada’s first game development hackathons with Red Bull Canada: redbull.com/adrenalan."
resume3 = "Experimented with state of the art NLP techniques to detect type of toxicity in online comments. Experimented with LSTMs, word embeddings (Glove, FastText), and techniques such as sequence bucketing. \
Fine-tuned and ensembled Bidirectional Encoder Representations from Transformers (BERT)."
resume4 = "Integrated an “auto-swipe” feature into Tinder based on real-time brain EEG data collected by the Muse. \
Tested on participants at Hack The 6ix 2017 and achieved a 75% accuracy."
resume5 = "Knowledgeable in: Git, Docker, Python (Pandas, Numpy, Scikit-Learn), Keras, Pytorch, Java, Javascript, VueJS, Firebase"

In [57]:
# tr4w = TextRank4Keyword()
# tr4w.analyze(resume1 + resume2 + resume3 + resume4 + resume5, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
# keywords = tr4w.get_keywords(50)

In [58]:
tr4w = TextRank4Keyword()
resume_sections = [resume1, resume2, resume3, resume4, resume5]
weights = [2, 1, 1, 1, 1]
total_keywords = []
for i, resume in enumerate(resume_sections):
    tr4w.analyze(resume, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
    keywords = tr4w.get_keywords(20)
    #modify weights
    for ix, kw in enumerate(keywords):
        keywords[ix][1] = keywords[ix][1] * weights[i]
    total_keywords += keywords

print(total_keywords)

[['ML', 3.6518648148148154], ['data', 3.4397268518518516], ['feasibility', 2.449461111111111], ['Jupyter', 2.3342388888888888], ['libraries', 2.2401407407407405], ['language', 2.2035907407407405], ['forecasting', 2.1711805555555554], ['supply', 2.1058722222222217], ['projects', 2.073761111111111], ['science', 1.921611111111111], ['processing', 1.8823851851851852], ['series', 1.8493217592592592], ['models', 1.8493217592592592], ['chain', 1.753122222222222], ['concept', 1.721011111111111], ['time', 1.4940532407407408], ['predictions', 1.4940532407407408], ['areas', 1.370811111111111], ['proof', 1.3602333333333334], ['interns', 1.3369999999999997], ['practices', 1.2972388888888888], ['Canada', 1.1678277777777777], ['hackathons', 1.1678277777777777], ['sponsorship', 1.0814583333333332], ['events', 1.0814583333333332], ['attendees', 1.0814583333333332], ['platform', 1.0], ['data', 1.0], ['analytics', 1.0], ['development', 0.993011111111111], ['Red', 0.993011111111111], ['worth', 0.8778125],

In [66]:
total_keywords

[['ML', 3.6518648148148154],
 ['data', 3.4397268518518516],
 ['feasibility', 2.449461111111111],
 ['Jupyter', 2.3342388888888888],
 ['libraries', 2.2401407407407405],
 ['language', 2.2035907407407405],
 ['forecasting', 2.1711805555555554],
 ['supply', 2.1058722222222217],
 ['projects', 2.073761111111111],
 ['science', 1.921611111111111],
 ['processing', 1.8823851851851852],
 ['series', 1.8493217592592592],
 ['models', 1.8493217592592592],
 ['chain', 1.753122222222222],
 ['concept', 1.721011111111111],
 ['time', 1.4940532407407408],
 ['predictions', 1.4940532407407408],
 ['areas', 1.370811111111111],
 ['proof', 1.3602333333333334],
 ['interns', 1.3369999999999997],
 ['practices', 1.2972388888888888],
 ['Canada', 1.1678277777777777],
 ['hackathons', 1.1678277777777777],
 ['sponsorship', 1.0814583333333332],
 ['events', 1.0814583333333332],
 ['attendees', 1.0814583333333332],
 ['platform', 1.0],
 ['data', 1.0],
 ['analytics', 1.0],
 ['development', 0.993011111111111],
 ['Red', 0.993011111

In [59]:
def score(keywords, job):
    tr4w.analyze(job, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
    job_keywords = tr4w.get_keywords(50)
    score = 0
    for w1 in keywords:
        for w2 in job_keywords:
            if w1[0] == w2[0]:
                score += w1[1]
    return score

In [60]:
import time
start = time.time()
scores = []
for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
    doc = doc1 + " " + doc2 + " " + doc3
    s = score(total_keywords, doc)
    scores.append((i, s))
print(time.time() - start)

68.20982241630554


In [61]:
scores = sorted(scores,key=lambda x: x[1], reverse=True)

In [64]:
scores

[(379, 17.81965972222222),
 (403, 17.083238425925924),
 (989, 16.167528703703702),
 (867, 15.54385972222222),
 (412, 15.032435185185184),
 (974, 15.024446759259257),
 (1245, 14.789719444444446),
 (777, 14.774538888888888),
 (1255, 14.773122222222222),
 (991, 14.649321296296295),
 (61, 14.540285185185184),
 (594, 14.492937037037038),
 (330, 14.04529537037037),
 (581, 14.04529537037037),
 (967, 13.992209722222222),
 (1242, 13.971534259259261),
 (444, 13.908917592592593),
 (469, 13.676662962962961),
 (161, 13.535350462962962),
 (767, 13.50431898148148),
 (120, 13.420923148148146),
 (480, 13.334263888888888),
 (1240, 13.246005555555554),
 (949, 13.123684259259258),
 (1133, 13.123684259259258),
 (813, 13.085961574074073),
 (1222, 13.05228425925926),
 (1158, 12.827844907407407),
 (561, 12.69953425925926),
 (1185, 12.67153935185185),
 (877, 12.660961574074074),
 (1208, 12.509544907407406),
 (78, 12.499384259259259),
 (465, 12.465093055555554),
 (950, 12.420923148148146),
 (80, 12.386923148148

In [65]:
for job in scores[:50]:
    X = job[0]
    print("ID: ", X)
    print(original["job_title"].values[X], ", ", original["organization"].values[X])
    print(original["job_responsibilities"].values[X])
    print(original["job_summary"].values[X])
    print(original["required_skills"].values[X])
    print("================================")
    print()

ID:  379
Actuarial Analyst ,  Northbridge (A Fairfax Company)
- Pricing, loss reserving, classification experience, and other pricing/reserving-related analysis;
- Compile and analyze company data for rate/reserve analysis;
- Assist in the preparation of rate fillings;
- Financial reporting, market research, and other ad hoc projects
Northbridge
Northbridge is one of Canada's leading commercial Property & Casualty insurance companies with $1.3 billion in premiums providing a wide range of insurance products including property, automobile and general liability to businesses primarily in Canada.  The Northbridge Group of Companies includes Northbridge Insurance which offers a broad array of innovative insurance solutions through our broker partners, Federated Insurance that operates through its dedicated sales agents, and Zenith which sells personal lines. Northbridge is a 100% Canadian owned company, and a wholly-owned subsidiary of Fairfax Financial Holdings.

Job Summary 
We are seeki

Transportation Engineering Co-op Student (Toronto West) ,  IBI Group Inc
- Assist on major transit/transportation studies, environmental assessments, and master plans involving the need and justification for new transportation projects 
- Alternatives analysis, demand forecasting, transportation system/network planning and operations, transportation master plans, and sustainable transportation 
- Perform technical analysis and assist with report writing to support studies 
- Involvement in traffic studies
Looking for a unique co-op/internship opportunity? 

If you're driven, enthusiastic and want to be part of an awesome team, keep reading... 

Students at IBI Group are given the opportunity to learn and grow from day one. We are always working on new and exciting projects for clients from around the world. Our goal is to have you join us, learn from us, and stay with us. 

IBI Group Co-op Students and Interns are eligible for the Campus Connext Award. This is an annual scholarship tha

In accordance with the Ontario Human Rights Code, Accessibility for Ontarians with Disabilities Act, 2005, and the Ontario Power Generation's Accessibility Standard, accommodation will be provided in all parts of the hiring process. Applicants need to make their needs known by contacting us in advance.
We are searching for an enthusiastic Engineering, Business or MBA student who has completed a minimum of 3 year(s) and will be returning to university or college (full time) upon completion of this work term or requires one final work term to graduate.
 
Preference may be given to the candidate who has the following skills and qualification:
Electrical Engineering background
Understanding of the energy sector
Experience with business development and project management
Experience supporting and collaborating with cross-functional teams
Strategic and analytical skills
Strong written and verbal communication skills
Self-motivated

This placement may involve some travel.  A valid G2 driver's

All other duties as may be assigned from time to time.
The Company
Pliteq Inc. is an international supplier of recycled products to the construction industry. We are a group of experienced engineers focused on developing high-performing, cost-effective acoustical products to ensure building code is met for sound transmission. Innovative by design, simple to install, GenieClip® and GenieMat® are the trusted brands of architects, builders and acoustical consultants worldwide.

Pliteq continues to be a leading innovator in vibration and sound control on a global scale. We have a competitive and entrepreneurial culture and are currently recruiting for several engineering special projects assistants to help with our growth.
The ideal candidate will be responsible and trustworthy, possess strong organizational and time management skills, detail orientated, excels in proofing and information verification, takes direction and follows established procedures, highly motivated and committed to wo

AWS
About You:
You are passionate about technology, and especially excited about exceeding customer expectations by delivering cutting-edge, web-facing solutions using the best technologies available. You are excited to write code and learn development processes in an Agile environment.

ID:  967
Quality Engineering ,  Exco Engineering
Job Description:
Exco Engineering is looking for a hands-on student to perform quality testing on large powertrain aluminum transmission castings.  The student will get experience in an industrial setting (casting foundry) working with saws, polishing equipment and air decay leak test fixture.

Duties will include:

Documenting casting process
Preparing test samples
Performing air decay leak test on castings
Tracking data
Preparing quality reports
Presenting results
About Exco Engineering:
Exco Engineering is a world leader in the design, build and development of large automotive high pressure die cast tooling, serving our powertrain, body and structural

...

-  Further experience and interest appreciated in MERN+J (Mongo, Express, ReactJS/Redux, NodeJS/Koa/ExpressJS, Static - Jekyll), LAMP+D (Linux,Apache, MySQL, PHP, CMS - Drupal); Invision, Slack, Jira, Confluence, GitHub, Docker, Apache Solr.

ID:  950
Admissions & Recruitment Associate ,  University of Waterloo
Assist with the planning, development and implementation of various student relations and recruitment events, such as but not limited to:
	
March Open House
You@Waterloo Day
Fellowship Interview Days
SAF Outreach and Ambassador Program
SAF Recruitment Tours
Warm & Fuzzy Letter Writing Campaign
DECA Provincial Prep Conference
DECA Provincial Competition
High school visits
DECA Leadership Conference


Performing marketing and recruitment administrative responsibilities such as:
	
Responding to prospective student enquiries
Reviewing and updating content for the SAF websites and faculty web profile pages
Assisting with content/story development and proofing of the SAF Year in Revie

Read more about Polar's Culture here: http://polar.me/blog/culture/polars-culture-built-foundation-great-workplace/
Primary Skills

Best practices of web software development
Web Development and Design using HTML, CSS and Javascript
Profiling websites using modern web development tools to optimize and troubleshoot scripts

It's a bonus (but not a requirement) if you're already familiar with some of the technologies and/or tools we use every day: 
 
-JQuery and Other JS Frameworks
-Chrome Web Inspector
-GitHub, Jira 
-Python, Django

ID:  376
Business Intelligence Developer / Consultant (HOME-BASED) ,  Iridium Technology
Since this is a home-based job, weekly sykpe and email correspondence is a must, as our team is located all over the world. 

The role encompasses a combination of consulting and development duties, including:
* Client-facing resource: work directly with law firm clients to implement and support Iridium BI products
* Report/dashboard design and development
* Manage clie

- Actively collaborate with information technology groups and stakeholders to support the delivery of organizational and information technology
The Ontario Teachers' Pension Plan (Ontario Teachers') is Canada's largest single-profession pension plan, with $175.6 billion in net assets at December 31, 2016.  It holds a diverse global portfolio of assets, approximately 80% of which is managed in-house, and has earned an average annualized rate of return of 10.1% since the plan's founding in 1990. Ontario Teachers' is an independent organization headquartered in Toronto. Its Asia-Pacific region office is located in Hong Kong and its Europe, Middle East & Africa region office is in London. The defined-benefit plan, which is fully funded, invests and administers the pensions of the province of Ontario's 318,000 active and retired teachers. For more information, visit otpp.com and follow us on Twitter @OtppInfo.

IT Service Delivery builds and manages the innovative systems that support the p

Note:  Interested candidates must demonstrate in a cover letter how they meet the qualification factors listed above.

ID:  1080
QA Mobile Apps ,  Ceridian
This position is to test both Android and iOS native app functionality.
Test new feature development as well as regression test existing functionality.
This is exploratory testing, test plan/case driven testing.
Exposure to automating testing is a bonus, not a requirement.
Dayforce HCM Mobile is an enterprise mobile app. We are currently averaging almost 4 million logins a week. And that volume is growing steadily.

We build, test and support a native iOS and native Android app. The app allows employees to manage their work life via mobile devices. From checking their schedule, to trading shifts, asking for a day off to checking their latest pay details.

This job is to work as a tester (QA) with our team to test both the iOS or Android mobile app.
Curiosity, diligence, attention to detail and a preference for mobile usage. The abil

-     Strong documentation & organizational skills

ID:  1012
8556-Commercial Banking Analyst - London ,  Scotiabank
KEY ACCOUNTABILITIES:
-Deepen client relationships by understanding clients' businesses, being a client champion, providing administrative support/service for client-facing interactions and providing actionable recommendations to support credit decisions
-Conduct company and industry analysis
-Conduct financial modelling (financial and credit analysis)
-Collect and assimilate information necessary to make sound credit decisions
-Make it easy to do business by minimizing touch points to provide fast turnaround times, including sound and timely credit underwriting support and financial analysis
Join the Global Community of Scotiabankers to help customers become financially better off.
Scotiabank is a premier financial institution and Canada's most international bank recognized as a Best Workplace®, in Canada, Mexico, El Salvador, Costa Rica, Puerto Rico, Dominican Republic